In [1]:
from typing import List


class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.lgh = ei - si + 1
        self.left = self.right = None
        self.s = 0  # 区间和
        self.mn = 0  # 区间最小值
        self.lazy = 0  # 该节点全部增加lazy 但未下发


class SegmentTree:

    def __init__(self, si, ei):
        self.root = Node(si, ei)

    def push_down(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy != 0:
            node.left.s += node.left.lgh * node.lazy
            node.right.s += node.right.lgh * node.lazy
            node.left.mn += node.lazy
            node.right.mn += node.lazy
            node.left.lazy += node.lazy
            node.right.lazy += node.lazy
            node.lazy = 0

    def update(self, node: Node, l, r, val):  # [l,r]区间全增加val
        if l <= node.si and node.ei <= r:
            node.s += node.lgh * val
            node.mn += val
            node.lazy += val
            return
        self.push_down(node)
        if node.mi >= l:
            self.update(node.left, l, r, val)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val)
        node.s = node.left.s + node.right.s
        node.mn = min(node.left.mn, node.right.mn)

    def left_idx(self, node: Node, r, val):  # [0,r]范围内<=val的最小下标
        if node.mn > val: return -1
        if node.si == node.ei:
            return node.si
        self.push_down(node)
        if node.left.mn <= val:
            return self.left_idx(node.left, l, r, val)
        if node.mi + 1 <= r:
            return self.left_idx(node.right, l, r, val)
        return -1

    def query(self, node: Node, l, r):  # [l,r]区间和
        if l <= node.si and node.ei <= r:
            return node.s
        self.push_down(node)
        ans = 0
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans


class BookMyShow:

    def __init__(self, n: int, m: int):
        self.lgh = n * m
        self.n = n
        self.m = m
        self.st = SegmentTree(0, self.lgh - 1)
        self.left = [m] * n  # 每一排剩余的座位数

    def gather(self, k: int, maxRow: int) -> List[int]:
        r = self.st.left_idx(self.st.root, maxRow, self.m - k)
        # print(r,maxRow)
        if r > maxRow: return []
        c = self.m - self.left[r]
        self.st.update(self.st.root, r * self.m + c, r * self.m + c + k - 1, 1)
        self.left[r] -= k
        return [r, c]

    def scatter(self, k: int, maxRow: int) -> bool:
        q = (maxRow + 1) * self.m - self.st.query(self.st.root, 0, (maxRow + 1) * self.m - 1)
        if q < k: return False
        r = 0
        while True:
            if self.left[r] >= k:
                c = self.m - self.left[r]
                self.left[r] -= k
                self.st.update(self.st.root, 0, r * self.m + c + k - 1, 1)
                return True
            k -= self.left[r]
            self.left[r] = 0
            r += 1


# Your BookMyShow object will be instantiated and called as such:
# obj = BookMyShow(n, m)
# param_1 = obj.gather(k,maxRow)
# param_2 = obj.scatter(k,maxRow)

In [ ]:
from collections import defaultdict


class SegmentTree:
    def __init__(self, n) -> None:
        self.s = defaultdict(int)
        self.lazy = defaultdict(lambda :False)
    def push_down(self, o, l, r, m):
        if self.lazy[o] is not None:
            self.s[o * 2] = (m - l + 1) * self.lazy[o]
            self.s[o * 2 + 1] = (r - m) * self.lazy[o]
            self.lazy[o * 2] = self.lazy[o]
            self.lazy[o * 2 + 1] = self.lazy[o]
            self.lazy[o] = None
    def update(self, o, l, r, L, R, val):
        if L <= l and r <= R:
            self.s[o] = (r - l + 1) * val
            self.lazy[o] = val
            return
        m = l + r >> 1
        self.push_down(o, l, r, m)
        if m >= L:
            self.update(o * 2, l, m, L, R, val)
        if m + 1 <= R:
            self.update(o * 2 + 1, m + 1, r, L, R, val)
        self.s[o] = self.s[o * 2] + self.s[o * 2 + 1]
    def query(self, o, l, r, L, R):
        if L <= l and r <= R:
            return self.s[o]
        m = l + r >> 1
        self.push_down(o, l, r, m)
        ans = 0
        if m >= L:
            ans += self.query(o * 2, l, m, L, R)
        if m + 1 <= R:
            ans += self.update(o * 2 + 1, m + 1, r, L, R)
        return ans

class BookMyShow:

    def __init__(self, n: int, m: int):
        self.lgh = n * m
        self.n = n
        self.m = m
        self.st = SegmentTree(self.lgh)
        self.left = [m] * n # 每一排剩余的座位数

    def gather(self, k: int, maxRow: int) -> List[int]:
        r = 0
        while r <= maxRow:
            if self.left[r] < k:
                r += 1
            else:
                break
        # print(r,maxRow)
        if r > maxRow: return []
        c = self.m - self.left[r]
        self.st.update(1, 0, self.lgh - 1, r * self.m + c, r * self.m + c + k - 1, 1)
        self.left[r] -= k
        return [r, c]

    def scatter(self, k: int, maxRow: int) -> bool:
        q = (maxRow + 1) * self.m - self.st.query(1, 0, self.lgh - 1, 0, (maxRow + 1) * self.m - 1)
        if q < k: return False
        r = 0
        while True:
            if self.left[r] >= k:
                c = self.m - self.left[r]
                self.left[r] -= k
                self.st.update(1, 0, self.lgh - 1, 0, r * self.m + c + k - 1, 1)
                return True
            k -= self.left[r]
            self.left[r] =   0
            r += 1



# Your BookMyShow object will be instantiated and called as such:
# obj = BookMyShow(n, m)
# param_1 = obj.gather(k,maxRow)
# param_2 = obj.scatter(k,maxRow)